In [0]:
sales_df = spark.read.option("header", "true").option("inferSchema", "true").csv("/Volumes/upload/default/files/sales_data.csv")
store_df = spark.read.option("header", "true").option("inferSchema", "true").csv("/Volumes/upload/default/files/store_data.csv")
display(sales_df.sample(fraction=0.25))
display(store_df.sample(fraction=0.5))

sale_id,store_id,product_id,sale_date,quantity,total_amount
sale_4,2.0,26,2022-06-25,30,231.85
sale_6,14.0,3,2022-08-20,83,522.16
sale_7,8.0,14,2021-06-12,31,1301.42
sale_10,15.0,45,2022-02-07,34,385.49
null,1.0,15,2020-10-27,28,1309.62
sale_30,18.0,11,2021-11-07,14,388.12
sale_35,null,34,2022-03-11,98,2600.47
sale_40,2.0,11,null,94,4293.04
sale_42,5.0,16,2021-02-03,74,1877.34
sale_52,17.0,21,2021-01-04,49,1244.01


store_id,store_region,store_size,open_date
1.0,West,2263.0,null
2.0,West,1510.0,2009-07-07
3.0,West,4989.0,2000-08-23
9.0,North,3653.0,2018-12-24
14.0,North,1116.0,2003-06-30
null,West,1968.0,2001-04-08
16.0,East,null,2003-10-01
17.0,East,null,2010-09-23
18.0,West,1191.0,2016-01-23


In [0]:
from pyspark.sql.functions import col, avg

store_df = store_df.filter(col("store_id").isNotNull())
#display(store_df.select(avg("store_size")).first()[0])
avg_store_size = store_df.select(avg("store_size")).first()[0]
store_df = store_df.fillna({"store_size": avg_store_size})
store_df = store_df.filter(col("open_date").isNotNull())
display(store_df.sample(fraction=0.5))


store_id,store_region,store_size,open_date
3.0,West,4989.0,2000-08-23
4.0,West,4942.0,2016-08-12
7.0,North,2606.2,2005-07-19
9.0,North,3653.0,2018-12-24
11.0,East,2606.2,2018-04-22
12.0,West,840.0,2000-03-22
13.0,West,2606.2,2009-10-03
16.0,East,2606.2,2003-10-01
17.0,East,2606.2,2010-09-23
18.0,West,1191.0,2016-01-23


In [0]:
from pyspark.sql.functions import col, to_date 

sales_df = sales_df.fillna({"quantity": 0, "total_amount": 0.0})
sales_df = sales_df.dropDuplicates()
sales_df = sales_df.filter(col("sale_id").isNotNull())
sales_df = sales_df.filter(col("store_id").isNotNull())
sales_df = sales_df.filter(col("sale_date").isNotNull())
sales_df = sales_df.filter((col("quantity") > 0) & (col("total_amount") > 0.0))

display(sales_df.sample(fraction=0.25))


sale_id,store_id,product_id,sale_date,quantity,total_amount
sale_878,17.0,13,2021-08-05,16,222.96
sale_22,16.0,14,2021-02-12,84,1848.9
sale_724,8.0,9,2020-11-01,44,1396.99
sale_169,5.0,50,2022-06-23,42,644.25
sale_591,14.0,40,2022-08-29,11,265.51
sale_30,18.0,11,2021-11-07,14,388.12
sale_265,19.0,3,2021-02-23,35,1631.11
sale_736,6.0,26,2020-06-20,59,2069.84
sale_906,13.0,14,2020-04-18,66,2396.55
sale_302,9.0,49,2022-06-19,91,3503.75


In [0]:
sales_store_df = sales_df.join(store_df, on="store_id", how="inner")
display(sales_store_df.sample(fraction=0.05))

store_id,sale_id,product_id,sale_date,quantity,total_amount,store_region,store_size,open_date
16.0,sale_22,14,2021-02-12,84,1848.9,East,null,2003-10-01
16.0,sale_871,44,2022-01-07,90,2001.19,East,null,2003-10-01
19.0,sale_607,25,2022-10-20,24,979.69,North,null,2018-10-12
14.0,sale_745,43,2022-11-27,32,1355.32,North,1116.0,2003-06-30
12.0,sale_740,19,2021-01-29,87,721.74,West,840.0,2000-03-22
14.0,sale_366,33,2020-02-20,25,454.06,North,1116.0,2003-06-30
12.0,sale_885,8,2021-07-18,21,1004.62,West,840.0,2000-03-22
16.0,sale_666,7,2021-07-22,18,550.62,East,null,2003-10-01
5.0,sale_674,30,2021-10-02,98,4202.83,North,null,2002-06-09
1.0,sale_610,41,2020-03-27,62,1167.38,West,2263.0,null


In [0]:
from pyspark.sql.functions import year, col, round

sales_store_df = sales_store_df.withColumn("sale_year", year(col("sale_date")))
sales_store_df = sales_store_df.filter(col("store_size").isNotNull())
sales_store_df = sales_store_df.withColumn("sales_per_sqft", round(col("total_amount") / col("store_size"), 2))
display(sales_store_df)

store_id,sale_id,product_id,sale_date,quantity,total_amount,store_region,store_size,open_date,sale_year,sales_per_sqft
14.0,sale_801,35,2021-10-17,62,367.17,North,1116.0,2003-06-30,2021,0.33
4.0,sale_441,20,2022-06-05,37,692.77,West,4942.0,2016-08-12,2022,0.14
12.0,sale_341,12,2021-09-29,18,202.03,West,840.0,2000-03-22,2021,0.24
8.0,sale_724,9,2020-11-01,44,1396.99,West,691.0,null,2020,2.02
20.0,sale_671,15,2022-09-30,82,710.78,North,4867.0,2019-11-25,2022,0.15
14.0,sale_591,40,2022-08-29,11,265.51,North,1116.0,2003-06-30,2022,0.24
2.0,sale_691,30,2022-08-15,34,660.9,West,1510.0,2009-07-07,2022,0.44
8.0,sale_204,26,2022-10-11,12,163.24,West,691.0,null,2022,0.24
18.0,sale_30,11,2021-11-07,14,388.12,West,1191.0,2016-01-23,2021,0.33
8.0,sale_388,32,2020-06-21,3,59.45,West,691.0,null,2020,0.09


In [0]:
sales_store_df.createOrReplaceTempView("sales_store")

In [0]:
sales_store_sql = spark.sql("""
    select 
        store_id, 
        store_region,
        round(sum(total_amount),0) as total_sales,
        sum(quantity) as total_quantity
    from sales_store
    group by store_id, store_region
""")
display(sales_store_sql)

store_id,store_region,total_sales,total_quantity
20.0,North,53057.0,2138
1.0,West,41750.0,1556
8.0,West,67746.0,2404
3.0,West,40980.0,1638
4.0,West,38285.0,1240
2.0,West,47392.0,1648
12.0,West,34735.0,1372
14.0,North,41333.0,1778
9.0,North,49838.0,1902
18.0,West,32691.0,1503


In [0]:
top_product_sql = spark.sql("""
                                select 
                                    product_id,
                                    sum(quantity) as total_quantity
                                from sales_store
                                group by product_id
                                order by total_quantity desc
                                limit 5
                            """)
display(top_product_sql)

product_id,total_quantity
15,736
26,634
32,592
28,592
13,552


In [0]:
sales_store_sql.createOrReplaceTempView("store_sales_summary")
display(spark.sql("select * from store_sales_summary"))

store_id,store_region,total_sales,total_quantity
20.0,North,53057.0,2138
1.0,West,41750.0,1556
8.0,West,67746.0,2404
3.0,West,40980.0,1638
4.0,West,38285.0,1240
2.0,West,47392.0,1648
12.0,West,34735.0,1372
14.0,North,41333.0,1778
9.0,North,49838.0,1902
18.0,West,32691.0,1503


In [0]:
top_store_sql = spark.sql("""
                        select store_id,
                        total_sales
                        from store_sales_summary
                        order by total_sales desc
                        limit 5
                    """)
display(top_store_sql)                    

store_id,total_sales
8.0,67746.0
20.0,53057.0
9.0,49838.0
2.0,47392.0
1.0,41750.0


In [0]:
top_product_sql.write.parquet("/Volumes/upload/default/files/top_product")
top_store_sql.write.parquet("/Volumes/upload/default/files/top_store")